In [ ]:
#@title Import necessary packages and set correct device

import clip
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.measure import regionprops
from skimage.segmentation import slic, watershed
from skimage.util import img_as_float
from torch.utils.data import random_split
from torchvision.ops import box_iou
from tqdm import tqdm

from modules.refcocog import RefCOCOg, RefCOCOgSample
from modules.yoloclip import display_preds

%matplotlib inline

if torch.cuda.is_available():
    device = torch.device("cuda")  # CUDA GPU
    print("[INFO] Using GPU.")
elif torch.has_mps:
    device = torch.device("mps")  # Apple Silicon GPU
    print("[INFO] Using MPS.")
else:
    device = torch.device("cpu")
    print("[INFO] No GPU found, using CPU instead.")


In [ ]:
#@title Import RefCOCOg dataset and its train/val/test splits

# data_path = "/media/dmmp/vid+backup/Data/refcocog"
data_path = "dataset/refcocog"

dataset = RefCOCOg(ds_path=data_path)

train_ds = RefCOCOg(ds_path=data_path, split='train')
val_ds = RefCOCOg(ds_path=data_path, split='val')
test_ds = RefCOCOg(ds_path=data_path, split='test')

# keep only a toy portion of each split
# keep = 0.1
# dataset, _ = random_split(dataset, [int(keep * len(dataset)), len(dataset) - int(keep * len(dataset))])
# train_ds, _ = random_split(train_ds, [int(keep * len(train_ds)), len(train_ds) - int(keep * len(train_ds))])
# val_ds, _ = random_split(val_ds, [int(keep * len(val_ds)), len(val_ds) - int(keep * len(val_ds))])
# test_ds, _ = random_split(test_ds, [int(keep * len(test_ds)), len(test_ds) - int(keep * len(test_ds))])

print(f"Dataset Size: {len(dataset)}\n")
print(f"Train size: {len(train_ds)}")
print(f"Val size:   {len(val_ds)}")
print(f"Test size:  {len(test_ds)}")


In [ ]:
#@title Utility function to downsample a map

def downsample_map(map, factor):
    # number of blocks in each dimension
    blocks_h = map.shape[0] // factor
    blocks_w = map.shape[1] // factor

    # reshape the original matrix into blocks
    blocks = map[:blocks_h * factor, :blocks_w * factor].reshape(blocks_h, factor, blocks_w, factor)

    # calculate the average of each block
    averages = blocks.mean(axis=(1, 3))

    return averages


In [ ]:
#@ Utility function to perform best bounding box search

def find_best_bbox(heatmap, show=False):
    heatmap = heatmap.astype(np.uint8)

    # Threshold the image to create a binary mask
    ret, mask = cv2.threshold(heatmap, heatmap.min(), heatmap.max(), cv2.THRESH_BINARY)

    # Find contours in the binary mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # print(f"[INFO] {len(contours)} detections.")

    best_score = 0
    best_bbox = None

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        score = heatmap[y:y + h, x:x + w].sum()
        if score > best_score:
            best_score = score
            best_bbox = (x, y, w, h)

    if show:
        # Display the map with the best bbox
        x, y, w, h = best_bbox
        rect = plt.Rectangle((x, y), w, h, linewidth=1, edgecolor=(0, 1, 0), facecolor='none')
        plt.gca().add_patch(rect)
        plt.imshow(map)
        plt.axis("off")
        plt.show()

    return best_bbox


In [ ]:
#@ Definition of Segmentation + CLIP visual grounding pipeline

class ClipSeg:

    def __init__(self, methods, n_segments=None, q=0.95, d=16, device=device, quiet=False):

        self.methods = list(methods)
        self.clip_model, self.clip_prep = clip.load("ViT-L/14", device=device)
        self.n_segments = n_segments
        self.q = q
        self.d = d
        self.device = device
        self.quiet = quiet

        supported_methods = ["s", "w"]

        for method in self.methods:
            if method not in supported_methods:
                raise ValueError(f"Method `{method}` not supported. Supported methods are: {supported_methods}")

    def _compute_hmap(self, np_image, prompt, method, masks):

        hmaps = list()

        for i, n in enumerate(masks):

            if method == "s":  # slic
                segments = slic(np_image, n_segments=n, compactness=10, sigma=1)
            elif method == "w":  # watershed
                segments = watershed(sobel(rgb2gray(np_image)), markers=n, compactness=0.001)

            regions = regionprops(segments)

            prompt_tkn = clip.tokenize(prompt).to(self.device)

            with torch.no_grad():
                prompt_enc = self.clip_model.encode_text(prompt_tkn)

            images_encs = list()

            pbar = tqdm(regions, position=0, leave=False) if not self.quiet else regions

            for region in pbar:

                if not self.quiet:
                    pbar.set_description(f"[INFO] method \"{method.upper()}\" "
                                         f"| mask {i + 1}/{len(self.n_segments)} "
                                         f"| computing CLIP scores",
                                         refresh=True)

                rect = region.bbox
                rect = (rect[1], rect[0], rect[3], rect[2])

                sub_image = sample.img.crop(rect)
                sub_image = self.clip_prep(sub_image).unsqueeze(0)
                sub_image = sub_image.to(self.device)

                with torch.no_grad():
                    image_enc = self.clip_model.encode_image(sub_image)

                images_encs.append(image_enc)

            images_encs = torch.cat(images_encs, dim=0)

            scores = prompt_enc @ images_encs.T

            scores = scores.squeeze().cpu().numpy()

            heatmap = np.zeros((segments.shape[0], segments.shape[1]))

            for i in range(segments.shape[0]):
                for j in range(segments.shape[1]):
                    heatmap[i, j] = scores[segments[i, j] - 1]

            hmaps.append(heatmap)

        return np.mean(np.array(hmaps), axis=0), hmaps

    def __call__(self, img_sample, prompt, show=False, show_pipeline=False, show_masks=False):

        if self.quiet:
            show = False
            show_pipeline = False

        # Convert image to np array
        np_image = img_as_float(io.imread(img_sample.path))

        # Apply all methods requested
        all_p_heatmaps = list()
        all_heatmaps = list()

        for method in self.methods:
            p_heatmap, heatmaps = self._compute_hmap(np_image, prompt, method, self.n_segments)
            all_p_heatmaps.append(p_heatmap)
            all_heatmaps.append(heatmaps)

        p_heatmap = np.mean(np.array(all_p_heatmaps), axis=0)

        # Shut down pixels below a certain threshold
        ths = np.quantile(p_heatmap.flatten(), self.q)
        fp_heatmap = p_heatmap.copy()
        fp_heatmap[p_heatmap < ths] = ths

        # Downsample the heatmap by a factor d
        dfp_heatmap = downsample_map(fp_heatmap, self.d)

        # Find the best bounding box
        pred_bbox = find_best_bbox(dfp_heatmap)

        if pred_bbox is None:
            return {"IoU": 0}

        if self.d > 1:
            pred_bbox = [pred_bbox[0] * self.d + self.d // 2,
                         pred_bbox[1] * self.d + self.d // 2,
                         pred_bbox[2] * self.d - self.d // 2,
                         pred_bbox[3] * self.d - self.d // 2]

        # Convert bbox format
        pred_bbox = [pred_bbox[0], pred_bbox[1], pred_bbox[2] + pred_bbox[0], pred_bbox[3] + pred_bbox[1]]

        # Get ground truth bbox
        gt_bbox = img_sample.bbox

        # Compute IoU
        iou = box_iou(
            torch.tensor([pred_bbox]),
            torch.tensor([gt_bbox])
        )

        # Show all computed masks, if requested

        if show_masks:
            for m in range(len(self.methods)):
                ax_cols = 4
                ax_rows = len(self.n_segments) // ax_cols + 1
                fig, axes = plt.subplots(ax_rows, ax_cols, figsize=(20, 5))
                for i, ax in enumerate(axes.ravel()):
                    ax.axis("off")
                    if i <= len(self.n_segments) - 1:
                        ax.imshow(all_heatmaps[m])

        # Show the mask processing pipeline, if requested
        if show_pipeline:
            fig, axes = plt.subplots(1, 4, figsize=(20, 5))

            for ax in axes.ravel():
                ax.axis("off")

            axes[0].imshow(np_image)
            axes[1].imshow(p_heatmap)
            axes[2].imshow(fp_heatmap)
            axes[3].imshow(dfp_heatmap)

        # Show the final prediction, if requested
        if show:
            display_preds(sample.img, prompt, pred_bbox, sample.bbox, f"{''.join(self.methods).upper()} + CLIP")

        return {"IoU": float(iou.item())}


In [ ]:
#@tile Test ClipSeg on a random sample

idx = np.random.randint(0, len(dataset))

sample = RefCOCOgSample(**dataset[idx])

clipseg = ClipSeg(methods="sw", n_segments=(16, 32, 64), q=0.9)

clipseg(sample, sample.sentences[0], show_pipeline=True, show_masks=False, show=True)


---

In [ ]:
#@title Function definition to test visual grounding with a given pipeline

def visual_grounding_test(vg_pipeline, dataset, track="IoU"):
    scores = list()

    pbar = tqdm(dataset)

    for sample in pbar:

        sample = RefCOCOgSample(**sample)

        for sentence in sample.sentences:
            sc = vg_pipeline(sample, sentence, show=False)

            scores.append(sc)

            avg_score = np.mean([score[track] for score in scores])

            pbar.set_description(f"Average {track}: {avg_score:.3f}")

    for metric in scores[0].keys():
        avg_metric = np.mean([score[metric] for score in scores])

        print("Avg. {}: {:.3f}".format(metric, avg_metric))


clipslic = ClipSeg(methods="s", n_segments=(8, 16, 32), q=0.9, quiet=True)

visual_grounding_test(clipslic, test_ds, track="IoU")
